## Books Analysis Using Spark SQL

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Books") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.csv("books.csv",header=True,inferSchema= True,sep=";");
df1 = spark.read.csv("ratings.csv",header=True,sep=";");

In [4]:
df.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [5]:
df1.show(5)

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
+-------+----------+-----------+
only showing top 5 rows



In [6]:
df.createOrReplaceTempView("books") 
df1.createOrReplaceTempView("ratings") 

In [7]:
df1.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)



### Highest number of books released per year

In [8]:
spark.sql("SELECT `Year-Of-Publication`,COUNT(*) as no_books FROM books GROUP BY `Year-Of-Publication` \
ORDER BY COUNT(*) DESC; ").show()

+-------------------+--------+
|Year-Of-Publication|no_books|
+-------------------+--------+
|               2002|   17628|
|               1999|   17432|
|               2001|   17360|
|               2000|   17235|
|               1998|   15767|
|               1997|   14892|
|               2003|   14359|
|               1996|   14031|
|               1995|   13548|
|               1994|   11796|
|               1993|   10602|
|               1992|    9906|
|               1991|    9389|
|               1990|    8661|
|               1989|    7937|
|               1988|    7493|
|               1987|    6529|
|               1986|    5841|
|               2004|    5839|
|               1985|    5343|
+-------------------+--------+
only showing top 20 rows



### List of recently published books

In [9]:
spark.sql("SELECT ISBN, `Book-Title`, `Book-Author`, `Year-Of-Publication`, `Publisher` FROM books \
WHERE `Year-Of-Publication` < 2020 ORDER BY `Year-Of-Publication` DESC; ").show()

+----------+--------------------+--------------------+-------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|
+----------+--------------------+--------------------+-------------------+--------------------+
|0307124533|Owl's Amazing but...|        Owl Magazine|               2012|        Golden Books|
|0394172116|Monkey (An Evergr...|         Cheng-En Wu|               2011|         Grove Press|
|0380816792|    A Rose in Winter|Kathleen E. Woodi...|               2011|Harper Mass Marke...|
|0743474198|        Heartbreaker|       Julie Garwood|               2010|              Pocket|
|0743457226|         Deep Waters|    Jayne Ann Krentz|               2010|              Pocket|
|074349346X|Angels &amp; Demo...|           Dan Brown|               2008|Washington Square...|
|0743448987|Memoirs of an Unf...|       Anne Robinson|               2006|        Pocket Books|
|1903436346|The Winter's Tale...|       

### List of oldest published books 

In [10]:
spark.sql("SELECT ISBN, `Book-Title`, `Book-Author`, `Year-Of-Publication`, `Publisher` FROM books \
WHERE `Year-Of-Publication` >0 ORDER BY `Year-Of-Publication` ; ").show() 

+----------+--------------------+--------------------+-------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|
+----------+--------------------+--------------------+-------------------+--------------------+
|964442011X|        Tasht-i khun|       Isma°il Fasih|               1376|      Nashr-i Alburz|
|9643112136|Dalan-i bihisht (...|         Nazi Safavi|               1378| Intisharat-i Quqnus|
|0781228956|Complete Works 10...|   Benjamin Franklin|               1806|Reprint Services ...|
|0781268001|Hugh Wynne, Free ...| Silas Weir Mitchell|               1897|Reprint Services ...|
|1551103982|The Cycling Adven...|         Ted Schredd|               1900|Graphic Arts Cent...|
|B00009NDBF|Help Yourself: Ce...|     David J. Pelzer|               1900|          E P Dutton|
|B0000VZEFK|Is That a Gun in ...|   Rachel Abramowitz|               1900|        Random House|
|0671407104|  LAST TEMP CHRISTCL|   Niko

### Most Popular Books

In [11]:
spark.sql("SELECT r.ISBN, `Book-Title`, count(*) as no_ratings FROM ratings r , books b WHERE r.ISBN = b.ISBN  \
 group by r.ISBN, `Book-Title` ORDER BY count(*) DESC; ").show() 

+----------+--------------------+----------+
|      ISBN|          Book-Title|no_ratings|
+----------+--------------------+----------+
|0971880107|         Wild Animus|      2502|
|0316666343|The Lovely Bones:...|      1295|
|0385504209|   The Da Vinci Code|       883|
|0060928336|Divine Secrets of...|       732|
|0312195516|The Red Tent (Bes...|       723|
|044023722X|     A Painted House|       647|
|0142001740|The Secret Life o...|       615|
|067976402X|Snow Falling on C...|       614|
|0671027360| Angels &amp; Demons|       586|
|0446672211|Where the Heart I...|       585|
|059035342X|Harry Potter and ...|       571|
|0316601950|The Pilot's Wife ...|       568|
|0375727345|House of Sand and...|       552|
|044021145X|            The Firm|       529|
|0452282152|Girl with a Pearl...|       526|
|0440214041|   The Pelican Brief|       523|
|0804106304|   The Joy Luck Club|       519|
|0440211727|      A Time to Kill|       517|
|0345337662|Interview with th...|       506|
|006093053

### Total Rating

In [12]:
spark.sql("SELECT r.ISBN, `Book-Title`, sum(r.`Book-Rating`) as total_ratings FROM ratings r , books b WHERE r.ISBN = b.ISBN  \
 group by r.ISBN, `Book-Title` ORDER BY sum(r.`Book-Rating`) DESC; ").show() 

+----------+--------------------+-------------+
|      ISBN|          Book-Title|total_ratings|
+----------+--------------------+-------------+
|0316666343|The Lovely Bones:...|       5787.0|
|0385504209|   The Da Vinci Code|       4108.0|
|0312195516|The Red Tent (Bes...|       3134.0|
|059035342X|Harry Potter and ...|       2798.0|
|0142001740|The Secret Life o...|       2595.0|
|0971880107|         Wild Animus|       2551.0|
|0060928336|Divine Secrets of...|       2524.0|
|0446672211|Where the Heart I...|       2402.0|
|0452282152|Girl with a Pearl...|       2219.0|
|0671027360| Angels &amp; Demons|       2179.0|
|044023722X|     A Painted House|       2062.0|
|0316601950|The Pilot's Wife ...|       2041.0|
|067976402X|Snow Falling on C...|       1999.0|
|0786868716|The Five People Y...|       1940.0|
|0446310786|To Kill a Mocking...|       1914.0|
|0743418174|         Good in Bed|       1899.0|
|0316769487|The Catcher in th...|       1868.0|
|043935806X|Harry Potter and ...|       

### Average Ratings 

In [13]:
spark.sql("SELECT r.ISBN, `Book-Title`, avg(r.`Book-Rating`) as avg_ratings,count(*) as no_ratings FROM ratings r , books b \
          WHERE r.ISBN = b.ISBN group by r.ISBN, `Book-Title` ORDER BY avg(r.`Book-Rating`) DESC, `Book-Title` ;").show()

+----------+--------------------+-----------+----------+
|      ISBN|          Book-Title|avg_ratings|no_ratings|
+----------+--------------------+-----------+----------+
|0399151788|        Dark Justice|       10.0|         1|
|0718143728|        "Blackadder"|       10.0|         1|
|0752201875|"Cracker": the Tr...|       10.0|         1|
|0310591635|        "Follow Me!"|       10.0|         1|
|0698301978|"I Can't" Said th...|       10.0|         1|
|1561705357|"Life Was Never M...|       10.0|         1|
|2277302910|"Soleil De Soufre...|       10.0|         1|
|0747235732|"The House of Cth...|       10.0|         1|
|0306811103|"Unsinkable": The...|       10.0|         1|
|0312113536|"You're Okay, It'...|       10.0|         1|
|0941925196|'California Histo...|       10.0|         1|
|0854098089|'Isms: a dictiona...|       10.0|         1|
|0842371907|'Round the Corner...|       10.0|         1|
|2266022253|006781: Bk.1 Gags...|       10.0|         1|
|0789447096|1,000 Makers of t..